In [11]:
import h5py
import imageio
import random
import numpy as np  
import pandas as pd 

from cellpose import models
from cellpose import utils as cp_utils
from cellstitch.utils import *
from cellstitch.pipeline import *
from cellstitch.evaluation import *

In [6]:
# filenames = set(get_filenames("../DATA/ATAS/raw/"))
# test_filenames = set(random.sample(filenames, int(0.3 * len(filenames))))
# train_filenames = filenames - test_filenames

In [24]:
dataset = "ATAS_5" # ATAS, ATAS_5, ATAS_10

In [8]:
filenames = set(get_filenames("../DATA/ATAS/raw/"))
train_filenames = []
for file in os.listdir("../DATA/ATAS/cellpose_train/"):
    if file.endswith(".npy_95.tif"):
        train_filenames.append(file[:-7])

test_filenames = filenames - set(train_filenames)

### PlantSeg 
- First, created a plantseg virtual enviroment: 
    - `conda install -c conda-forge mamba` 
    - `mamba create -n plant-seg -c pytorch -c nvidia -c conda-forge -c lcerrone plantseg pytorch-cuda=11.7` 
- activate the environment: `conda activate plant-seg` 
- download the ovules test dataset: https://osf.io/uzq3w/ to `../DATA/<dataset>/plantseg_test/` 
- set the `path` in `config.yaml` to `../DATA/<dataset>/plantseg_test/` 
- perform segmentation with the `generic_confocal_3D_unet` by running `plantseg --config config.yaml`

In [25]:
plantseg_results_folder = "../DATA/%s/plantseg_test/PreProcessing/generic_confocal_3D_unet/MultiCut" % dataset

In [26]:
for test_filename in test_filenames: 
    print("Starting %s" % test_filename) 
    
    with h5py.File("%s/%s_predictions_multicut.h5" % (plantseg_results_folder, test_filename[:-4]), "r") as f:
        plantseg = np.array(list(f['segmentation'])) 
        
    plantseg[np.where(plantseg == 1)] = 0 # plantseg use 1 as labels
    np.save("./results/%s/plantseg/%s" % (dataset, test_filename), plantseg)

Starting 68hrs_plant1_trim-acylYFP.npy
Starting 8hrs_plant2_trim-acylYFP.npy
Starting 4hrs_plant15_trim-acylYFP.npy
Starting 32hrs_plant13_trim-acylYFP.npy
Starting 28hrs_plant1_trim-acylYFP.npy
Starting 84hrs_plant18_trim-acylYFP.npy
Starting 76hrs_plant15_trim-acylYFP.npy
Starting 48hrs_plant15_trim-acylYFP.npy
Starting 44hrs_plant2_trim-acylYFP.npy
Starting 0hrs_plant2_trim-acylYFP.npy
Starting 20hrs_plant13_trim-acylYFP.npy
Starting 0hrs_plant18_trim-acylYFP.npy
Starting 24hrs_plant15_trim-acylYFP.npy
Starting 8hrs_plant18_trim-acylYFP.npy
Starting 12hrs_plant2_trim-acylYFP.npy
Starting 44hrs_plant4_trim-acylYFP.npy
Starting 40hrs_plant2_trim-acylYFP.npy
Starting 48hrs_plant4_trim-acylYFP.npy
Starting 36hrs_plant15_trim-acylYFP.npy
Starting 36hrs_plant2_trim-acylYFP.npy
Starting 20hrs_plant2_trim-acylYFP.npy
Starting 24hrs_plant13_trim-acylYFP.npy
Starting 44hrs_plant18_trim-acylYFP.npy
Starting 8hrs_plant15_trim-acylYFP.npy
Starting 60hrs_plant2_trim-acylYFP.npy
Starting 8hrs_plan

### Train cellpose model from scratch

In [ ]:
# generate training data
data_folder = "../DATA/ATAS"
cellpose_folder = "../DATA/ATAS/cellpose_train"

In [ ]:
for train_filename in train_filenames: 
    img = np.load("%s/raw/%s" % (data_folder, train_filename))
    labels = np.load("%s/labels/%s" % (data_folder, train_filename)) 
    depth = img.shape[0] 
    
    for i in range(depth): 
        imageio.imwrite("%s/%s_%s.tif" % (cellpose_folder, train_filename, i), img[i])
        imageio.imwrite("%s/%s_%s_masks.tif" % (cellpose_folder, train_filename, i), labels[i])

`python -m cellpose --train --use_gpu --dir ../DATA/ATAS/cellpose_train --pretrained_model None --n_epochs 100  --verbose`

### Generate cellpose3d results

In [9]:
model_dir = '../DATA/ATAS/cellpose_train/models/cellpose_residual_on_style_on_concatenation_off_cellpose_train_2023_04_28_14_46_49.933105'

In [10]:
flow_threshold = 1
model = models.CellposeModel(gpu=True, pretrained_model=model_dir)

In [ ]:
for test_filename in test_filenames: 
    print("Starting %s" % test_filename)
    img = np.load("../DATA/%s/raw/%s" % (dataset, test_filename)) 
    masks, _, _ = model.eval(img, do_3D=True, flow_threshold=flow_threshold, channels = [0,0]) 
    np.save("./results/%s/cellpose3d/%s" % (dataset, test_filename), masks)

### Generate cellpose2d results

In [11]:
for test_filename in test_filenames: 
    print("Starting %s" % test_filename)
    img = np.load("../DATA/%s/raw/%s" % (dataset, test_filename)) 
    masks, _, _ = model.eval(list(img), do_3D=False, flow_threshold=flow_threshold, channels = [0,0])
    masks = cp_utils.stitch3D(np.array(masks))
    
    np.save("./results/%s/cellpose2d/%s" % (dataset, test_filename), masks)

Starting 76hrs_plant15_trim-acylYFP.npy
Starting 48hrs_plant15_trim-acylYFP.npy
Starting 4hrs_plant15_trim-acylYFP.npy
Starting 60hrs_plant4_trim-acylYFP.npy
Starting 24hrs_plant13_trim-acylYFP.npy
Starting 36hrs_plant2_trim-acylYFP.npy
Starting 0hrs_plant13_trim-acylYFP.npy
Starting 72hrs_plant18_trim-acylYFP.npy
Starting 20hrs_plant2_trim-acylYFP.npy
Starting 36hrs_plant15_trim-acylYFP.npy
Starting 72hrs_plant2_trim-acylYFP.npy
Starting 68hrs_plant1_trim-acylYFP.npy
Starting 40hrs_plant13_trim-acylYFP.npy
Starting 76hrs_plant1_trim-acylYFP.npy
Starting 44hrs_plant2_trim-acylYFP.npy
Starting 44hrs_plant4_trim-acylYFP.npy
Starting 12hrs_plant2_trim-acylYFP.npy
Starting 64hrs_plant13_trim-acylYFP.npy
Starting 8hrs_plant15_trim-acylYFP.npy
Starting 84hrs_plant18_trim-acylYFP.npy
Starting 60hrs_plant2_trim-acylYFP.npy
Starting 40hrs_plant2_trim-acylYFP.npy
Starting 0hrs_plant18_trim-acylYFP.npy
Starting 12hrs_plant18_trim-acylYFP.npy
Starting 8hrs_plant18_trim-acylYFP.npy
Starting 44hrs_p

### Generate cellstitch results

In [12]:
for test_filename in test_filenames: 
    print("Starting %s" % test_filename)
    img = np.load("../DATA/%s/raw/%s" % (dataset, test_filename))
    
    cellstitch, _, _ = model.eval(list(img), flow_threshold=flow_threshold, channels = [0,0])
    cellstitch = np.array(cellstitch)

    yz_masks, _, _ = model.eval(list(img.transpose(1,0,2)), flow_threshold=flow_threshold, channels = [0,0])
    yz_masks = np.array(yz_masks).transpose(1,0,2)

    xz_masks, _, _ = model.eval(list(img.transpose(2,1,0)), flow_threshold=flow_threshold, channels = [0,0])
    xz_masks = np.array(xz_masks).transpose(2,1,0)

    full_stitch(cellstitch, yz_masks, xz_masks)
    
    np.save("./results/%s/cellstitch/%s" % (dataset, test_filename), cellstitch) 

Starting 76hrs_plant15_trim-acylYFP.npy


Starting 48hrs_plant15_trim-acylYFP.npy


Starting 4hrs_plant15_trim-acylYFP.npy


Starting 60hrs_plant4_trim-acylYFP.npy


Starting 24hrs_plant13_trim-acylYFP.npy


...

Starting 80hrs_plant15_trim-acylYFP.npy


Starting 48hrs_plant4_trim-acylYFP.npy


Starting 40hrs_plant1_trim-acylYFP.npy


Starting 20hrs_plant13_trim-acylYFP.npy


Starting 24hrs_plant15_trim-acylYFP.npy


# Benchmark Results

In [27]:
methods = ["cellstitch", "cellpose2d", "cellpose3d", "plantseg"]
 
for method in methods: 
    print("Starting %s" % method) 
    
    data = []
    for filename in test_filenames:
        print("Starting %s" % filename)
        labels = np.load('../DATA/%s/labels/%s' % (dataset, filename)) 
        masks = np.load("./results/%s/%s/%s" % (dataset, method, filename))
        
        ap25, _, _, _ = average_precision(labels, masks, 0.25)
        ap50, tp, fp, fn = average_precision(labels, masks, 0.5)
        ap75, _, _, _ = average_precision(labels, masks, 0.75) 
        
        if (tp + fp) != 0:
            precision = tp / (tp + fp)
        else: 
            precision = 0
        if (tp + fn) != 0:
            recall = tp / (tp + fn)
        else:
            recall = 0

        row = [ 
            filename, 
            ap25,
            ap50,
            ap75,
            tp, 
            fp, 
            fn, 
            precision,
            recall
        ]

        data.append(row)

    df = pd.DataFrame(data, columns=[
        "filename",
        "ap25", 
        "ap50",
        "ap75", 
        "tp", 
        "fp", 
        "fn",
        "precision",
        "recall"
    ])

    df.to_csv("./results/%s/%s.csv" % (dataset, method), index=False)

Starting plantseg
Starting 68hrs_plant1_trim-acylYFP.npy
Starting 8hrs_plant2_trim-acylYFP.npy
Starting 4hrs_plant15_trim-acylYFP.npy
Starting 32hrs_plant13_trim-acylYFP.npy
Starting 28hrs_plant1_trim-acylYFP.npy
Starting 84hrs_plant18_trim-acylYFP.npy
Starting 76hrs_plant15_trim-acylYFP.npy
Starting 48hrs_plant15_trim-acylYFP.npy
Starting 44hrs_plant2_trim-acylYFP.npy
Starting 0hrs_plant2_trim-acylYFP.npy
Starting 20hrs_plant13_trim-acylYFP.npy
Starting 0hrs_plant18_trim-acylYFP.npy
Starting 24hrs_plant15_trim-acylYFP.npy
Starting 8hrs_plant18_trim-acylYFP.npy
Starting 12hrs_plant2_trim-acylYFP.npy
Starting 44hrs_plant4_trim-acylYFP.npy
Starting 40hrs_plant2_trim-acylYFP.npy
Starting 48hrs_plant4_trim-acylYFP.npy
Starting 36hrs_plant15_trim-acylYFP.npy
Starting 36hrs_plant2_trim-acylYFP.npy
Starting 20hrs_plant2_trim-acylYFP.npy
Starting 24hrs_plant13_trim-acylYFP.npy
Starting 44hrs_plant18_trim-acylYFP.npy
Starting 8hrs_plant15_trim-acylYFP.npy
Starting 60hrs_plant2_trim-acylYFP.npy
